In [41]:
import pandas as pd
import altair as alt
import numpy as np

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [42]:
#/scratch/ucgd/lustre-labs/marth/scratch/u0746015/COLO829/intersections/rufus_exclusive_long_read_isecs/five_way/

In [43]:
# Add headers
num_samples = 6
sample_columns = [f"File{i+1}_DP" for i in range(num_samples)]

# Read data without a header and assign column names
column_names = ['CHROM', 'POS'] + sample_columns
depth_df = pd.read_csv('/Users/genetics/Documents/code/altair/colo829/blood_long_reads/blood_only_five_way_5per_plus_ALT_sample_depths.tsv.tsv', sep='\t', header=None, names=column_names)

# Create a unique variant ID (could also combine CHROM_POS)
depth_df['variant_id'] = depth_df['CHROM'].astype(str) + ':' + depth_df['POS'].astype(str)

# Extract sample DP columns (assuming they start at 3rd column)
dp_columns = depth_df.columns[2:-1]  # or simply 2: if variant_id is not yet present
if 'variant_id' in depth_df.columns:
    dp_columns = depth_df.columns[2:-1]  # CHROM, POS, ..., variant_id
else:
    dp_columns = depth_df.columns[2:]


# Compute summary metrics
# IF THIS LINE BREAKS IT'S DUE TO . INSTEAD OF 0s (just find and replace in file)
depth_df['avg_coverage'] = depth_df[dp_columns].mean(axis=1)
depth_df['stddev_coverage'] = depth_df[dp_columns].std(axis=1)
depth_df['num_samples_found'] = (depth_df[dp_columns] > 0).sum(axis=1)

# Select output columns
summary_df = depth_df[['variant_id', 'avg_coverage', 'stddev_coverage', 'num_samples_found']]

# Save to TSV
summary_df.to_csv('temp_variant_summary.tsv', sep='\t', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/genetics/Documents/code/altair/colo829/blood_long_reads/blood_only_five_way_5per_plus_ALT_sample_depths.tsv.tsv'

In [ ]:
# Load data from TSV file
data = pd.read_csv('/Users/genetics/Documents/code/altair/colo829/long_reads/temp_variant_summary.tsv', sep='\t')
data = data.apply(pd.to_numeric, errors='ignore')  # Convert numeric columns to appropriate types

# Error bounds
data['lower_bound'] = data['avg_coverage'] - data['stddev_coverage']
data['upper_bound'] = data['avg_coverage'] + data['stddev_coverage']

# Define color scale (adjust to your needs) 
color_scale = alt.Scale(
    domain=[0, 6],  # Adjust this range based on your data
    range=['yellow', 'royalblue']
)

error_bars = alt.Chart(data).mark_errorbar(clip=True, color='lightgray').encode(
    x=alt.X('variant_id:N', title='Variant ID', axis=None),
    y=alt.Y('lower_bound:Q', title='Average Coverage'),
    y2=alt.Y2('upper_bound:Q', title='Average Coverage'),
)

# Create points with error bars
points = alt.Chart(data).mark_point(filled=True, size=20, clip=True).encode(
    x=alt.X('variant_id:N', title='Variant ID', axis=None),
    y=alt.Y('avg_coverage:Q', title='Average Coverage'),
    color=alt.Color('num_samples_found:Q', scale=color_scale, legend=alt.Legend(title='# Files Found In')),
    tooltip=['variant_id', 'avg_coverage', 'stddev_coverage', 'num_samples_found']
)

# Combine layers
chart =  error_bars + points

chart = chart.properties(
    width=1200,
    height=600,
    title='Long Read Average Coverage for 5%+ VAF, 5-Way Sites With ALT Allele in Blood-Only Admixture'
).configure_title(fontSize=24).configure_axis(labelFontSize=16, titleFontSize=16).configure_legend(labelFontSize=14, titleFontSize=14)

chart


/var/folders/gq/tmwqn0b1225dz42d9gps_lqr0000gn/T/ipykernel_85338/1715372848.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.apply(pd.to_numeric, errors='ignore')  # Convert numeric columns to appropriate types


alt.LayerChart(...)

In [128]:
# Bar chart for number of variants in sample bins (exclusive)

def categorize_samples(n):
    if n >= 16:
        return '16 samples'
    elif n >= 12:
        return '12+ samples'
    elif n >= 8:
        return '8+ samples'
    elif n >= 4:
        return '4+ samples'
    elif n >= 2:
        return '2+ samples'
    else:
        return '<2 samples'
    
data['exclus_samp_cat'] = data['num_samples_found'].apply(categorize_samples)
category_counts = data.groupby('exclus_samp_cat').size().reset_index(name='variant_count')
category_order = ['2+ samples', '4+ samples', '8+ samples', '12+ samples', '16 samples']
category_counts['sample_category'] = pd.Categorical(category_counts['exclus_samp_cat'], categories=category_order, ordered=True)

# Bar chart
bar_chart = alt.Chart(category_counts).mark_bar().encode(
    x=alt.X('exclus_samp_cat:N', title='Sample Bins (Exclusive)', sort=category_order),
    y=alt.Y('variant_count:Q', title='Number of Variants'),
    color=alt.Color('sample_category:N', legend=None)
).properties(
    width=400,
    height=300,
    title='Variant Counts Binned by Number of Long-Read Samples They Were Found In (Exclusive)'
)

# Text labels
text = alt.Chart(category_counts).mark_text(
    align='center',
    baseline='bottom',
    dy=-5
).encode(
    x=alt.X('exclus_samp_cat:N', sort=category_order),
    y=alt.Y('variant_count:Q'),
    text=alt.Text('variant_count:Q')
)

# Combine (simple + operator for two Chart objects)
chart_with_labels = bar_chart + text

# Config
chart_with_labels = chart_with_labels.configure_title(fontSize=18).configure_axis(labelFontSize=12, titleFontSize=14)

chart_with_labels


alt.LayerChart(...)

In [116]:
# AF charts from total sample counts


# BLOOD ONLY
# Add headers
num_samples = 6
sample_columns = [f"File{i+1}_DP" for i in range(num_samples)]
column_names = ['CHROM', 'POS'] + sample_columns
ref_depth_df = pd.read_csv('/Users/genetics/Documents/code/altair/colo829/blood_long_reads/blood_only_five_way_5per_plus_REF_sample_depths.tsv', sep='\t', header=None, names=column_names)

# Create a unique variant ID (could also combine CHROM_POS)
ref_depth_df['variant_id'] = ref_depth_df['CHROM'].astype(str) + ':' + ref_depth_df['POS'].astype(str)

# Extract sample DP columns (assuming they start at 3rd column)
dp_columns = ref_depth_df.columns[2:-1]  # or simply 2: if variant_id is not yet present
if 'variant_id' in ref_depth_df.columns:
    dp_columns = ref_depth_df.columns[2:-1]  # CHROM, POS, ..., variant_id
else:
    dp_columns = ref_depth_df.columns[2:]

# Compute summary metrics
# IF THIS LINE BREAKS IT'S DUE TO . INSTEAD OF 0s (just find and replace in file)
ref_depth_df['ref_coverage'] = ref_depth_df[dp_columns].sum(axis=1)
ref_depth_df['ref_samples_found'] = (ref_depth_df[dp_columns] > 0).sum(axis=1)

# Read ALT data
alt_depth_df = pd.read_csv('/Users/genetics/Documents/code/altair/colo829/blood_long_reads/blood_only_five_way_5per_plus_ALT_sample_depths.tsv', sep='\t', header=None, names=column_names)

# Create a unique variant ID (could also combine CHROM_POS)
alt_depth_df['variant_id'] = alt_depth_df['CHROM'].astype(str) + ':' + alt_depth_df['POS'].astype(str)

# Extract sample DP columns (assuming they start at 3rd column)
dp_columns = alt_depth_df.columns[2:-1]  # or simply 2: if variant_id is not yet present
if 'variant_id' in alt_depth_df.columns:
    dp_columns = alt_depth_df.columns[2:-1]  # CHROM, POS, ..., variant_id
else:
    dp_columns = alt_depth_df.columns[2:]

# Compute summary metrics
# IF THIS LINE BREAKS IT'S DUE TO . INSTEAD OF 0s (just find and replace in file)
alt_depth_df['alt_coverage'] = alt_depth_df[dp_columns].sum(axis=1)
alt_depth_df['alt_samples_found'] = (alt_depth_df[dp_columns] > 0).sum(axis=1)

merged_blood = pd.merge(ref_depth_df, alt_depth_df, on='variant_id', how='left', suffixes=('_refblood', '_altblood'))
merged_blood['alt_coverage'] = merged_blood['alt_coverage'].fillna(0)
count_ALT = merged_blood['alt_coverage']
count_REF = merged_blood['ref_coverage']
count_TOTAL = count_ALT + count_REF
merged_blood['total_coverage'] = count_TOTAL
merged_blood['AF'] = count_ALT / count_TOTAL
merged_blood['source'] = 'blood'
merged_blood = merged_blood.rename(columns={'alt_coverage': 'alt', 'total_coverage': 'total'})  

merged_blood.to_csv('merged_blood.tsv', sep='\t', index=False)

In [117]:

# BLT50 DATA
num_samples = 16
sample_columns = [f"File{i+1}_DP" for i in range(num_samples)]
column_names = ['CHROM', 'POS'] + sample_columns
ref_blt50_depth_df = pd.read_csv('/Users/genetics/Documents/code/altair/colo829/long_reads/REF_only_5way_per_sample_depth.tsv', sep='\t', header=None, names=column_names)

# Create a unique variant ID (could also combine CHROM_POS)
ref_blt50_depth_df['variant_id'] = ref_blt50_depth_df['CHROM'].astype(str) + ':' + ref_blt50_depth_df['POS'].astype(str)

# Extract sample DP columns (assuming they start at 3rd column)
dp_columns = ref_blt50_depth_df.columns[2:-1]  # or simply 2: if variant_id is not yet present
if 'variant_id' in ref_blt50_depth_df.columns:
    dp_columns = ref_blt50_depth_df.columns[2:-1]  # CHROM, POS, ..., variant_id
else:
    dp_columns = ref_blt50_depth_df.columns[2:]

# Compute summary metrics
# IF THIS LINE BREAKS IT'S DUE TO . INSTEAD OF 0s (just find and replace in file)
ref_blt50_depth_df['ref_coverage'] = ref_blt50_depth_df[dp_columns].sum(axis=1)
ref_blt50_depth_df['ref_samples_found'] = (ref_blt50_depth_df[dp_columns] > 0).sum(axis=1)

# Read ALT data
alt_blt50_depth_df = pd.read_csv('/Users/genetics/Documents/code/altair/colo829/long_reads/ALT_5way_per_sample_depth.tsv', sep='\t', header=None, names=column_names)

# Create a unique variant ID (could also combine CHROM_POS)
sample_columns = [f"File{i+1}_DP" for i in range(0,15)]
column_names = ['CHROM', 'POS'] + sample_columns
alt_blt50_depth_df['variant_id'] = alt_blt50_depth_df['CHROM'].astype(str) + ':' + alt_blt50_depth_df['POS'].astype(str)

# Extract sample DP columns (assuming they start at 3rd column)
dp_columns = alt_blt50_depth_df.columns[2:-1]  # or simply 2: if variant_id is not yet present
if 'variant_id' in alt_blt50_depth_df.columns:
    dp_columns = alt_blt50_depth_df.columns[2:-1]  # CHROM, POS, ..., variant_id
else:
    dp_columns = alt_blt50_depth_df.columns[2:]

# Compute summary metrics
# IF THIS LINE BREAKS IT'S DUE TO . INSTEAD OF 0s (just find and replace in file)
alt_blt50_depth_df['alt_coverage'] = alt_blt50_depth_df[dp_columns].sum(axis=1)
alt_blt50_depth_df['alt_samples_found'] = (alt_blt50_depth_df[dp_columns] > 0).sum(axis=1)

merged_blt50 = pd.merge(ref_blt50_depth_df, alt_blt50_depth_df, on='variant_id', how='left', suffixes=('_refblt50', '_altblt50'))
merged_blt50['alt_coverage'] = merged_blt50['alt_coverage'].fillna(0)
count_ALT = merged_blt50['alt_coverage']
count_REF = merged_blt50['ref_coverage']
count_TOTAL = count_ALT + count_REF
merged_blt50['total_coverage'] = count_TOTAL
merged_blt50['AF'] = count_ALT / count_TOTAL
merged_blt50['source'] = 'blt50'
merged_blt50 = merged_blt50.rename(columns={'alt_coverage': 'alt', 'total_coverage': 'total'})


In [132]:
combined = pd.concat([merged_blood[['variant_id', 'alt', 'total', 'AF', 'source']],
                      merged_blt50[['variant_id', 'alt', 'total', 'AF', 'source']]])

In [178]:
chrom = 'chr9'
filtered = combined[combined['variant_id'].str.startswith(chrom)]  # Filter for AF > 0.05

pivot = combined.pivot(index='variant_id', columns='source', values='AF').reset_index()
pivot['blood_greater_blt50'] = pivot['blood'] > pivot['blt50']
pivot['diff_magnitude'] = (pivot['blt50'] - pivot['blood']).abs()

#combined_flagged = combined.merge(pivot[['variant_id', 'blood_greater_blt50']], on='variant_id')
combined_flagged = combined.merge(
    pivot[['variant_id', 'blood_greater_blt50', 'diff_magnitude']],
    on='variant_id'
)
# Rank the variants by diff magnitude
pivot_sorted = pivot.sort_values(by='diff_magnitude', ascending=False).reset_index(drop=True)
pivot_sorted['x_order'] = pivot_sorted.index

# Merge x_order back into combined
combined_flagged = combined_flagged.merge(
    pivot_sorted[['variant_id', 'x_order']],
    on='variant_id'
)



line_chart = alt.Chart(combined_flagged).mark_line().encode(
    x=alt.X('x_order:O', title='Variant ID (sorted by difference)', axis=None),
    y=alt.Y('AF:Q', title='Allele Frequency (Alt Count/Total Coverage)'),
    detail='variant_id:N',
    color=alt.condition(
        alt.datum.blood_greater_blt50,
        alt.value('yellow'),  # Highlight lines where blood > blt50
        alt.value('lightgray')     # Default line color
))

# Overlay the original points
points = alt.Chart(combined_flagged).mark_point(filled=True).encode(
    x=alt.X('x_order:O', title='Variant ID (sorted by difference)', axis=None),
    y=alt.Y('AF:Q', title='Allele Frequency (Alt Count/Total Coverage)'),
    color=alt.Color('source:N', scale=alt.Scale(domain=['blood', 'blt50'], range=['salmon', 'steelblue']), legend=alt.Legend(title='Source')),
    tooltip=['variant_id', 'AF', 'source']
).properties(
    width=3600,
    height=800, 
    title='Allele Frequency by Variant in COLO829 BLT50 & Blood'
)

combined_chart = line_chart + points
combined_chart

alt.LayerChart(...)

In [137]:

chart = alt.Chart(combined).mark_point().encode(
    x=alt.X('variant_id:N', title='Variant ID', axis=None),
    y=alt.Y('AF:Q', title='Allele Frequency (Alt Count/Total Coverage)'),
    color=alt.Color('source:N', scale=alt.Scale(domain=['blood', 'blt50'], range=['salmon', 'steelblue']), legend=alt.Legend(title='Source')),
    tooltip=['variant_id', 'alt', 'total', 'AF', 'source']
).properties(
    width=1200,
    height=800,
    title='Allele Frequency by Variant in COLO829 BLT50 & Blood'
)

chart

alt.Chart(...)